In [2]:
#do the essential imports
from __future__ import absolute_import, division, print_function
import os
import tensorflow as tf
import keras
tf.__version__

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


'1.0.1'

In [4]:
#laod and preprocess the data
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


11501568/11490434 [==============================] - 50s 4us/step


In [6]:
#create a simple model
# Returns a short sequential model
def create_model():
    model = keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
    model.compile(optimizer=keras.optimizers.Adam(), 
                loss=keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
    return model
# Create a basic model instance
model = create_model()
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [9]:
#saving checkpoints during training
# keras.callbacks.ModelCheckpoint is the callback that does this job
checkpoint_path = "../training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create checkpoint callback
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)
model = create_model()
model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 635us/step - loss: 1.1573 - acc: 0.6750 - val_loss: 0.6930 - val_acc: 0.7950

Epoch 00001: saving model to ../training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 1s 613us/step - loss: 0.4169 - acc: 0.8890 - val_loss: 0.5328 - val_acc: 0.8340

In [10]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 155us/step
Untrained model, accuracy:  7.90%


In [11]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 80us/step
Restored model, accuracy: 87.30%


In [14]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "../training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=10)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)



Epoch 00010: saving model to ../training_2/cp-0010.ckpt

Epoch 00020: saving model to ../training_2/cp-0020.ckpt

Epoch 00030: saving model to ../training_2/cp-0030.ckpt

Epoch 00040: saving model to ../training_2/cp-0040.ckpt

Epoch 00050: saving model to ../training_2/cp-0050.ckpt


In [21]:
#save the full model
model = create_model()
model.fit(train_images, train_labels, epochs=5)
# Save entire model to a HDF5 file
model.save('my_model.h5')


Epoch 1/5
1000/1000 [==============================] - 1s 620us/step - loss: 1.1527 - acc: 0.6830
Epoch 2/5
1000/1000 [==============================] - 0s 496us/step - loss: 0.4006 - acc: 0.9010

In [22]:
#recreate the model from same file
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()


TypeError: softmax() got an unexpected keyword argument 'axis'